<a href="https://colab.research.google.com/github/peremartra/Apress_LLProjects_Book/blob/main/4-Evaluating%20LLMs/4_3_evaluating_rag_giskard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div>
    <h1>Large Language Models Projects</a></h1>
    <h3>Apply and Implement Strategies for Large Language Models</h3>
    <h2>4.3-Evaluating LLMs with other LLMs.
  </h2>
    <h3>Evaluating a RAG solution with Giskard</h3>
    <p>by <b>Pere Martra</b></p>
</div>





In the final part of this notebook, you will find the necessary code to evaluate the suitability of the responses provided by the Agent using Giskard.

#Installing libraries & Loading Dataset

In [1]:
!pip install -q langchain==0.1.14
!pip install -q langchain-openai==0.1.1
!pip install langchainhub==0.1.15
#!pip install --upgrade -q tiktoken==0.5.2
!pip install -q datasets==2.16.1
!pip install -q chromadb==0.4.22

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 812.8/812.8 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 36.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.8/122.8 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!pip install -q openai==1.30.3

We will download the dataset from the Hugging Face datasets library. It's a dataset with information about diseases.

In [4]:
from datasets import load_dataset

data = load_dataset("keivalya/MedQuad-MedicalQnADataset", split='train')


Generating train split: 0 examples [00:00, ? examples/s]

In [5]:
data = data.to_pandas()
data.head(10)

,qtype,Question,Answer
0,susceptibility,Who is at risk for Lymphocytic Choriomeningiti...,LCMV infections can occur after exposure to fr...
1,symptoms,What are the symptoms of Lymphocytic Choriomen...,LCMV is most commonly recognized as causing ne...
2,susceptibility,Who is at risk for Lymphocytic Choriomeningiti...,Individuals of all ages who come into contact ...
3,exams and tests,How to diagnose Lymphocytic Choriomeningitis (...,"During the first phase of the disease, the mos..."
4,treatment,What are the treatments for Lymphocytic Chorio...,"Aseptic meningitis, encephalitis, or meningoen..."
5,prevention,How to prevent Lymphocytic Choriomeningitis (L...,LCMV infection can be prevented by avoiding co...
6,information,What is (are) Parasites - Cysticercosis ?,Cysticercosis is an infection caused by the la...
7,susceptibility,Who is at risk for Parasites - Cysticercosis? ?,Cysticercosis is an infection caused by the la...
8,exams and tests,How to diagnose Parasites - Cysticercosis ?,"If you think that you may have cysticercosis, ..."
9,treatment,What are the treatments for Parasites - Cystic...,Some people with cysticercosis do not need to ...


In [ ]:
data = data[0:100]

As you can see, the medical information in the dataset is well-organized, and to someone like me, who is not an expert in the field, it appears to be quite valuable. This information could be a useful addition to any general medicine book to support primary care doctors.

Load the langchain libraries to load the document.

In [6]:
from langchain.document_loaders import DataFrameLoader
from langchain.vectorstores import Chroma

The Document is in the Answer column, and the others columns are Metadata.

In [7]:
df_loader = DataFrameLoader(data, page_content_column="Answer")


In [8]:
df_document = df_loader.load()
display(df_document[:2])

[Document(page_content='LCMV infections can occur after exposure to fresh urine, droppings, saliva, or nesting materials from infected rodents.  Transmission may also occur when these materials are directly introduced into broken skin, the nose, the eyes, or the mouth, or presumably, via the bite of an infected rodent. Person-to-person transmission has not been reported, with the exception of vertical transmission from infected mother to fetus, and rarely, through organ transplantation.', metadata={'qtype': 'susceptibility', 'Question': 'Who is at risk for Lymphocytic Choriomeningitis (LCM)? ?'}),
 Document(page_content='LCMV is most commonly recognized as causing neurological disease, as its name implies, though infection without symptoms or mild febrile illnesses are more common clinical manifestations. \n                \nFor infected persons who do become ill, onset of symptoms usually occurs 8-13 days after exposure to the virus as part of a biphasic febrile illness. This initial 

We can chunk the documents. The size to which we want to split the document is a design decision. The larger it is, the larger the prompt will be, and the slower the Model's response process.

We also need to consider the maximum prompt size and ensure that the document does not exceed it.

In [9]:
from langchain.text_splitter import CharacterTextSplitter

In [10]:
text_splitter = CharacterTextSplitter(chunk_size=1250, chunk_overlap=100)
texts = text_splitter.split_documents(df_document)


These warnings we see are because it can't perform the partition of the required size. This is because it waits for a page break to divide the text and does so when possible.

In [11]:
first_doc = texts[1]
print(first_doc.page_content)

LCMV is most commonly recognized as causing neurological disease, as its name implies, though infection without symptoms or mild febrile illnesses are more common clinical manifestations. 
                
For infected persons who do become ill, onset of symptoms usually occurs 8-13 days after exposure to the virus as part of a biphasic febrile illness. This initial phase, which may last as long as a week, typically begins with any or all of the following symptoms: fever, malaise, lack of appetite, muscle aches, headache, nausea, and vomiting. Other symptoms appearing less frequently include sore throat, cough, joint pain, chest pain, testicular pain, and parotid (salivary gland) pain. 
                
Following a few days of recovery, a second phase of illness may occur. Symptoms may consist of meningitis (fever, headache, stiff neck, etc.), encephalitis (drowsiness, confusion, sensory disturbances, and/or motor abnormalities, such as paralysis), or meningoencephalitis (inflammation 

### Initialize the Embedding Model and Vector DB

We load the text-embedding-ada-002 model from OpenAI.

In [12]:
from getpass import getpass
import os
if not 'OPENAI_API_KEY' in os.environ:
  os.environ["OPENAI_API_KEY"] = getpass("OpenAI API Key: ")

OpenAI API Key: ··········


In [13]:
from langchain_openai import OpenAIEmbeddings

model_name = 'text-embedding-ada-002'
#model_name = 'text-embedding-3-small'

embed = OpenAIEmbeddings(
    model=model_name,
    #openai_api_key=OPENAI_API_KEY
)

The execution of this cell may take 3 to 5 minutes. If you want it to be faster, you can reduce the number of records in the dataset.

In [ ]:
directory_cdb = '/content/drive/MyDrive/chromadb'
chroma_db = Chroma.from_documents(
    df_document, embed, persist_directory=directory_cdb
)

We are going to create three objects.

* The language model, which can be any of those from OpenAI, the most common being gpt-3.5.
* The memory, responsible for keeping the prompt with all the necessary history.
* The retrieval, used to obtain information stored in ChromaDB.

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain_openai import OpenAI
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.chains import RetrievalQA

llm=OpenAI(#openai_api_key=OPENAI_API_KEY,
           temperature=0.0)

conversational_memory = ConversationBufferWindowMemory(
    memory_key='chat_history',
    k=4, #Number of messages stored in memory
    return_messages=True #Must return the messages in the response.
)

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=chroma_db.as_retriever()
)

We can try the isolated Retrieval to see if the information it returns is relevant.




In [ ]:
qa.run("What is the main symptom of LCM?")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


' The main symptom of LCM is a biphasic febrile illness, which includes symptoms such as fever, malaise, lack of appetite, muscle aches, headache, nausea, and vomiting.'

Perfect! The information returned is exactly what we desired.

## Creating the Agent.

In [ ]:
from langchain.agents import Tool, AgentExecutor
DESCRIPTION = 'use this tool when answering medical knowledge queries to get more information about the topic'
#DESCRIPTION = 'Use this tool if any part of the QUERY or CHAT HISTORY history discusses concepts related to medicine or diseases to get more information about the topic'

#Defining the list of tool objects to be used by LangChain.
tools = [
    Tool(
        name='Medical KB',
        func=qa.run,
        description=(
            DESCRIPTION
        )
    )
]

In [ ]:
from langchain.agents import initialize_agent, create_react_agent
from langchain import hub

prompt = hub.pull("hwchase17/react-chat")
agent = create_react_agent(
    #agent='chat-conversational-react-description',
    tools=tools,
    llm=llm,
    prompt=prompt,
    #verbose=True,
    #max_iterations=3,
    #early_stopping_method='generate'
)

In [ ]:
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent,
                               tools=tools,
                               verbose=True,
                               memory=conversational_memory,
                               max_iterations=30,
                               max_execution_time=600,
                               #early_stopping_method='generate',
                               handle_parsing_errors=True
                               )

### Using the Conversational Agent

To make queries we simply call the `agent` directly.

First i will try a order not related to the Medical field.

In [ ]:
agent_executor.invoke({"input": "Give me the area of square of 2x2"})



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? No
Final Answer: The area of a square with sides of 2x2 is 4 square units.

> Finished chain.


{'input': 'Give me the area of square of 2x2',
 'chat_history': [HumanMessage(content='What are the primary risk factors for acquiring this?'),
  AIMessage(content='The primary risk factors for acquiring Acanthamoeba keratitis include wearing contact lenses, having a compromised immune system, and exposure to contaminated water or soil.'),
  HumanMessage(content='What would be the best method?'),
  AIMessage(content='The best method for treating parasitic worm infections would be to consult with a medical professional and follow their recommended treatment plan.'),
  HumanMessage(content='Could you tell me what the procedure is?'),
  AIMessage(content='The procedure for treating parasitic worm infections would involve consulting with a medical professional and following their recommended treatment plan.'),
  HumanMessage(content='What is generally the basis for this?'),
  AIMessage(content='The basis for treating parasitic worm infections is to consult with a medical professional and f

Perfect, the model has responded without accessing the configured knowledge database.

Now I will try with a question that is also not related to health.

In [ ]:
agent_executor.invoke({"input": "Do you know who is Clark Kent?"})



> Entering new AgentExecutor chain...

Thought: Do I need to use a tool? No
Final Answer: Clark Kent is a fictional character known as Superman in the DC Comics universe. He is a superhero with superhuman abilities and a secret identity as a journalist.

> Finished chain.


{'input': 'Do you know who is Clark Kent?',
 'chat_history': [HumanMessage(content='What would be the best method?'),
  AIMessage(content='The best method for treating parasitic worm infections would be to consult with a medical professional and follow their recommended treatment plan.'),
  HumanMessage(content='Could you tell me what the procedure is?'),
  AIMessage(content='The procedure for treating parasitic worm infections would involve consulting with a medical professional and following their recommended treatment plan.'),
  HumanMessage(content='What is generally the basis for this?'),
  AIMessage(content='The basis for treating parasitic worm infections is to consult with a medical professional and follow their recommended treatment plan.'),
  HumanMessage(content='Give me the area of square of 2x2'),
  AIMessage(content='The area of a square with sides of 2x2 is 4 square units.')],
 'output': 'Clark Kent is a fictional character known as Superman in the DC Comics universe. He

It has not accessed either, as the model has been able to identify that it is not a question related to the database that LangChain provides.

Now it's time to try with a question related to Medicine. Let's see if the model can understand that it should first look for information in the vector database at its disposal.

In [ ]:
 agent_executor.memory.clear()

In [ ]:
agent_executor.invoke({"input": """I have a patient that can have Botulism,
how can I confirm the diagnose?"""})



> Entering new AgentExecutor chain...

Thought: Do I need to use a tool? Yes
Action: Medical KB
Action Input: Botulism Botulism is a rare but serious paralytic illness caused by a nerve toxin produced by certain bacteria. It can be contracted through contaminated food, wounds, or ingestion of bacterial spores. Symptoms include muscle paralysis, difficulty swallowing, and respiratory failure. Treatment includes antitoxin, supportive care, and removal of contaminated food or wound.Do I need to use a tool? No
Final Answer: To confirm a diagnosis of botulism, a doctor may perform a physical exam, review symptoms and medical history, and order laboratory tests such as a stool or blood test. They may also use imaging tests such as an MRI or CT scan to look for signs of nerve damage. It is important to seek medical attention immediately if botulism is suspected, as it can be life-threatening if left untreated.

> Finished chain.


{'input': 'I have a patient that can have Botulism,\nhow can I confirm the diagnose?',
 'chat_history': [],
 'output': 'To confirm a diagnosis of botulism, a doctor may perform a physical exam, review symptoms and medical history, and order laboratory tests such as a stool or blood test. They may also use imaging tests such as an MRI or CT scan to look for signs of nerve damage. It is important to seek medical attention immediately if botulism is suspected, as it can be life-threatening if left untreated.'}

Perfect, the most important thing for us is that it has been able to identify that it should go to the medical database to search for information about the symptoms.

In [ ]:
agent_executor.invoke({"input": "Is this an important illness?"})



> Entering new AgentExecutor chain...

Thought: Do I need to use a tool? No
Final Answer: Yes, botulism is a serious illness that can be life-threatening if left untreated. It is important to seek medical attention immediately if it is suspected.

> Finished chain.


{'input': 'Is this an important illness?',
 'chat_history': [HumanMessage(content='I have a patient that can have Botulism,\nhow can I confirm the diagnose?'),
  AIMessage(content='To confirm a diagnosis of botulism, a doctor may perform a physical exam, review symptoms and medical history, and order laboratory tests such as a stool or blood test. They may also use imaging tests such as an MRI or CT scan to look for signs of nerve damage. It is important to seek medical attention immediately if botulism is suspected, as it can be life-threatening if left untreated.')],
 'output': 'Yes, botulism is a serious illness that can be life-threatening if left untreated. It is important to seek medical attention immediately if it is suspected.'}

And the memory works perfectly. We can maintain a conversation, taking into account that the model knows the previous questions and answers.

## Evaluating the solution with Giskard

Install and Load the libraries.

In [ ]:
!pip install -q giskard[llm]
from giskard.rag import KnowledgeBase, generate_testset, evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 569.3/569.3 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 48.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 54.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.3/143.3 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.5/118.5 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.1/267.1 kB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 51.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 8.7 MB/

Is necesary to create a Dataframe with just the column containing the information used to create the RAG system.

In [ ]:
import pandas as pd
df_giskard = pd.DataFrame([d.page_content for d in df_document], columns=["text"])
df_giskard.head()

,text
0,LCMV infections can occur after exposure to fr...
1,LCMV is most commonly recognized as causing ne...
2,Individuals of all ages who come into contact ...
3,"During the first phase of the disease, the mos..."
4,"Aseptic meningitis, encephalitis, or meningoen..."


Using the information from the dataset, we ask Giskard to create a Knowledge Base, which is nothing more than a set of questions along with their respective answers. Both the questions and answers are generated by OpenAI's most advanced model, which is why it requires our OpenAI key to be provided.

In [ ]:
kb_giskard = KnowledgeBase(df_giskard)

In [ ]:
kb_giskard

In [ ]:
# The more questions you generate, the more you will be charged.
test_questions = generate_testset(
    kb_giskard,
    num_questions=30,
    agent_description="Medical assistant for diagnosis and treatment support.",
)

INFO:giskard.rag:Finding topics in the knowledge base.
INFO:giskard.rag:Computing Knowledge Base embeddings.
INFO:giskard.rag:Found 3 topics in the knowledge base.


Generating questions:   0%|          | 0/30 [00:00<?, ?it/s]

In [ ]:
df_test_questions = test_questions.to_pandas()

In [ ]:
df_test_questions.head()

,question,reference_answer,reference_context,conversation_history,metadata
id,,,,,
d7ab758a-4084-4c14-a641-ebe884d47f20,Which age group is most at risk of contracting...,School-age children who live in these areas ar...,"Document 24: Schistosomiasis, also known as bi...",[],"{'question_type': 'simple', 'seed_document_id'..."
51fe5624-993d-4cb8-9884-bc0cf5bdc4b2,How long does the yellow fever vaccine last an...,The yellow fever vaccine lasts for 10 years. T...,Document 23: If you continue to live or travel...,[],"{'question_type': 'simple', 'seed_document_id'..."
a6bc5067-b1a6-4169-b394-9f5e3371045f,What are the recommended preventive measures a...,Preventive measures include avoiding fruit bat...,Document 89: People who have close contact wit...,[],"{'question_type': 'simple', 'seed_document_id'..."
f9280f12-73c7-4d0b-9985-d0a759523ec2,What are some measures to prevent Toxocara inf...,"Some measures include treating pets, especiall...",Document 44: Controlling Toxocara infection in...,[],"{'question_type': 'simple', 'seed_document_id'..."
abbce36a-5b60-43e8-8e09-5d396f78c2b4,What measures can be taken to prevent Baylisas...,Baylisascaris infection can be prevented by av...,Document 66: Baylisascaris worms are intestina...,[],"{'question_type': 'simple', 'seed_document_id'..."


In [ ]:
df_test_questions['question'].iloc[1]


'How long does the yellow fever vaccine last and when does the International Certificate of Vaccination become valid?'

In [ ]:
df_test_questions['reference_answer'].iloc[1]


'The yellow fever vaccine lasts for 10 years. The International Certificate of Vaccination becomes valid 10 days after vaccination and lasts for 10 years.'

In [ ]:
df_test_questions['reference_context'].iloc[1]

'Document 23: If you continue to live or travel in yellow fever-endemic areas, you should receive a booster dose of yellow fever vaccine after 10 years.\n                \nAfter receiving the vaccine, you should receive an International Certificate of Vaccination (yellow card) that has been validated by the vaccination center. This Certificate becomes valid 10 days after vaccination and lasts for 10 years. You will need this card as proof of vaccination to enter certain countries.'

A function is created that will be called from Giskard's evaluate function. This function receives the question and returns the agent's response.

In [ ]:
def use_agent(question, history=None):
    return agent_executor.invoke({"input": question})

In [ ]:
agent_executor.memory.clear()

In [ ]:
report = evaluate(use_agent, testset=test_questions, knowledge_base=kb_giskard)

Asking questions to the agent:   0%|          | 0/30 [00:00<?, ?it/s]



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Medical KB
Action Input: Schistosomiasis Schistosomiasis is a disease caused by parasitic worms that is most commonly found in areas with poor sanitation. It is transmitted through contact with contaminated freshwater and can cause illness in humans. There is no vaccine available, but the best way to prevent schistosomiasis is to avoid swimming or wading in freshwater in affected areas and to drink safe water. Treatment is available with prescription medication. Do I need to use a tool? No
Final Answer: The age group most at risk of contracting schistosomiasis in areas with poor sanitation would likely be children and young adults who are more likely to come into contact with contaminated water sources.

> Finished chain.


> Entering new AgentExecutor chain...

Thought: Do I need to use a tool? Yes
Action: Medical KB
Action Input: Yellow fever vaccine After receiving the vaccine, you should receive a

Correctness evaluation:   0%|          | 0/30 [00:00<?, ?it/s]

In [ ]:
# Summary with the results.
report.correctness_by_question_type()

,correctness
question_type,
complex,1.0
conversational,0.0
distracting element,0.6
double,0.8
simple,1.0
situational,0.4


In [ ]:
report.correctness_by_topic()

,correctness
topic,
Botulism and Marine Toxin Poisoning,0.333333
Parasitic Infections and Prevention,0.666667


In [ ]:
report.component_scores()

,score
RAG Components,
GENERATOR,0.760000
RETRIEVER,0.800000
REWRITER,0.466667
ROUTING,1.000000
KNOWLEDGE_BASE,0.666667


In [ ]:
# Obtaining the incorrect answers
failures = report.get_failures()[:2]
failures

,question,reference_answer,reference_context,conversation_history,metadata,agent_answer,correctness,correctness_reason
id,,,,,,,,
f4c8d2a8-7894-4b4f-9818-45e941f29190,Considering the annual statistics of seafood p...,Eating raw or undercooked beef or pork is the ...,Document 7: Cysticercosis is an infection caus...,[],"{'question_type': 'distracting element', 'seed...",{'input': 'Considering the annual statistics o...,False,The agent incorrectly stated that consuming un...
3adeb291-59f0-44a7-93b6-8f00193983e9,What are the typical laboratory abnormalities ...,"During the first phase of the disease, the mos...",Document 3: During the first phase of the dise...,[],"{'question_type': 'distracting element', 'seed...",{'input': 'What are the typical laboratory abn...,False,The agent's answer does not match the ground t...


In [ ]:
# Giskard explains the reasons why it considers the answers to be incorrect.
failures['correctness_reason'].iloc[1]

"The agent's answer does not match the ground truth. The agent incorrectly states that anemia and elevated liver enzymes are common in the first phase of Babesia infection, and that hemolytic anemia and low platelet count are common in the second phase. The ground truth states that leukopenia and thrombocytopenia are common in the first phase, and that an increase in protein levels, an increase in the number of white blood cells or a decrease in the glucose levels in the CSF are common in the second phase."


# Conclusions.
The experiment has been a small success. The Vectorial database has been configured and filled with information from the dataset. A LangChain agent has been created, and it has been able to retrieve information from the database only when necessary. Don't forget that our ChatBot has memory.

All of this in just a few lines of code!


---